In [2]:
# Import 
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [15]:
# BSoup sample 
URL = "https://realpython.github.io/fake-jobs/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")   # Return bs4.BeautifulSoup

In [ ]:
# Cheatsheet
# ----------
URL = "https://realpython.github.io/fake-jobs/"
page = requests.get(URL)
# ----------

# Return bs4.BeautifulSoup
soup = BeautifulSoup(page.content, "html.parser")

# Find HTML Element -> Return bs4.element.Tag
results = soup.find(
    name="input",
    id="ResultsContainer",
) 

# print(soup.prettify())

# Find sub elements in element
job_cards = soup.find_all("div", class_="card-content")

# Pass a function to a bs4 method 
python_jobs = results.find_all(
    "h2", string=lambda text: "python" in text.lower()
);

# Fetch links in the job boards 
# If you used .text then it will leave only visable part, so only text is left 
# So the link you want in href is stripped as well. Do the following instead
for job_card in job_cards:
    link_url = job_card.find_all("a")[1]["href"]

#### 2. Sample with HousingTarget

- bs4 is a library to navigate documents 
- requests is a library for well, requesting but mostly to work with http session 
- MechanicalSoup is a Python library for automating interaction with websites

In [62]:
# Import 
from selenium import webdriver
import mechanicalsoup
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import sys
import pathlib
import importlib
import requests 

sys.path.append(str(pathlib.Path.cwd().parent))

import scraper
scraper = importlib.reload(scraper)

In [ ]:
# Get browser object 
url = "https://www.housingtarget.com/netherlands"
browser = mechanicalsoup.Browser()
page = browser.get(url)

# Get html 
html = page.soup

form = html.select("form")[0]
form.find("input", {"name": "LeftMenu_ZipCodes"})["value"] = "Utretch"

In [ ]:
import mechanicalsoup

# Create a browser object
browser = mechanicalsoup.StatefulBrowser()

# Open the webpage
browser.open("https://www.housingtarget.com/netherlands")

browser.select_form()

form = browser.get_current_form()
form["LeftMenu_LPEstateTypes"] = ["2", "3", "9", "20"]
form["LeftMenu_ZipCodes"] = "1319"

response = browser.submit_selected()

In [ ]:
# Set up Chrome options
chrome_option = Options()
chrome_option.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_option)

# Open the page after the form is submitted
driver.get(
    # 'https://www.housingtarget.com/netherlands/housing-rentals?' + 
    # 'zip_codes=1039;1319&estate_types=2;3;9;20&area_to=69&max_rent=2000'
    'https://www.housingtarget.com/netherlands/housing-rentals/amsterdam'
)

ul_element = driver.find_element(By.CLASS_NAME, 'table-ads')
li_elements = ul_element.find_elements(By.TAG_NAME, 'li')
listings = [li for li in li_elements if li.get_attribute("class") == "" and li.find_elements(By.TAG_NAME, "div")]

html = BeautifulSoup(driver.find_element(By.CLASS_NAME, "pager").get_attribute("outerHTML"), "html.parser")
urls = [e["href"] for e in html.find_all("a")] 
parsed_listings = [scraper.parse_listing_html(li) for li in listings]
r = pd.DataFrame(parsed_listings, columns = ["link", "rent", "size", "property_type", "location"])


In [ ]:
scraper.listing_spider("https://www.housingtarget.com/netherlands/housing-rentals/amsterdam")

Visiting https://www.housingtarget.com/netherlands/housing-rentals/amsterdam
Visiting https://www.housingtarget.com/netherlands/housing-rentals/amsterdam/pageindex2
Visiting https://www.housingtarget.com/netherlands/housing-rentals/amsterdam/pageindex3
Visiting https://www.housingtarget.com/netherlands/housing-rentals/amsterdam/pageindex4


#### Get details of listings
-----

In [57]:
url = 'https://www.housingtarget.com/rooms/utrecht-overvecht/12485098'

def get_listing_attr(url):
    try:
        page = requests.get(url, timeout=10)
        page.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None, None, None, None

    soup = BeautifulSoup(page.content, "html.parser")

    html = soup.find("div", class_="top-info-mobile")
    if not html:
        return None, None, None, None    
    
    title = html.findChild("h1").text
    street = html.findChild("strong").text
    description = soup.find_all("div", {"class" : "desc"})[0].text
    seo = soup.find_all("div", {"class" : "seo"})[0].text

    return title, street, description, seo

page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")    

#### Get Place based on query 
-------------------